In [2]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from google.colab import drive
import joblib

from numpy import mean
from numpy import std
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_selection import RFE

drive.mount('/content/gdrive')

path_Algol = '/content/gdrive/MyDrive/Doktora/Python/Algol'
path_Algol_Noisy = '/content/gdrive/MyDrive/Doktora/Python/Algol_Noisy'
Algol_files = [os.path.join(path_Algol, f) for f in os.listdir(path_Algol) if f.endswith('.txt')]
Algol_files = Algol_files + [os.path.join(path_Algol_Noisy, f) for f in os.listdir(path_Algol_Noisy) if f.endswith('.txt')]
data_Algol = []
labels_Algol = []

for file in Algol_files:
  labels_Algol.append("Algol")
  fileContent = []
  with open(file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            # Satırları ayrıştırın ve x, y koordinatlarını depolayın
            x, y = line.strip().split('\t')
            fileContent.append(float(y))
  data_Algol.append(fileContent)

path_Beta_Lyrae = '/content/gdrive/MyDrive/Doktora/Python/Beta_Lyrae'
path_Beta_Lyrae_Noisy = '/content/gdrive/MyDrive/Doktora/Python/Beta_Lyrae_Noisy'
Beta_Lyrae_files = [os.path.join(path_Beta_Lyrae, f) for f in os.listdir(path_Beta_Lyrae) if f.endswith('.txt')]
Beta_Lyrae_files = Beta_Lyrae_files + [os.path.join(path_Beta_Lyrae_Noisy, f) for f in os.listdir(path_Beta_Lyrae_Noisy) if f.endswith('.txt')]
data_Beta_Lyrae = []
labels_Beta_Lyrae = []

for file in Beta_Lyrae_files:
  labels_Beta_Lyrae.append("B Lyr")
  fileContent = []
  with open(file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            # Satırları ayrıştırın ve x, y koordinatlarını depolayın
            x, y = line.strip().split('\t')
            fileContent.append(float(y))
  data_Beta_Lyrae.append(fileContent)

path_W_UMa = '/content/gdrive/MyDrive/Doktora/Python/W_UMa_Noisy'
path_W_UMa_Noisy = '/content/gdrive/MyDrive/Doktora/Python/W_UMa_Noisy'
W_UMa_files = [os.path.join(path_W_UMa, f) for f in os.listdir(path_W_UMa) if f.endswith('.txt')]
W_UMa_files = W_UMa_files + [os.path.join(path_W_UMa_Noisy, f) for f in os.listdir(path_W_UMa_Noisy) if f.endswith('.txt')]
data_W_UMa = []
labels_W_UMa = []

for file in W_UMa_files:
  labels_W_UMa.append("W UMa")
  fileContent = []
  with open(file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            # Satırları ayrıştırın ve x, y koordinatlarını depolayın
            x, y = line.strip().split('\t')
            fileContent.append(float(y))
  data_W_UMa.append(fileContent)

ModuleNotFoundError: No module named 'google.colab'

In [2]:
# Verileri ve etiketleri numpy dizilerine dönüştürün
X = np.array(data_Algol+data_Beta_Lyrae+data_W_UMa)
y = np.array(labels_Algol+labels_Beta_Lyrae+labels_W_UMa)

X = X.astype('float')
y = LabelEncoder().fit_transform(y.astype('str'))


In [1]:
# transforms for the feature union
transforms = list()
transforms.append(('mms', MinMaxScaler()))
transforms.append(('ss', StandardScaler()))
transforms.append(('rs', RobustScaler()))
transforms.append(('qt', QuantileTransformer(n_quantiles=100, output_distribution='normal')))
transforms.append(('kbd', KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')))
transforms.append(('pca', PCA(n_components=7)))
transforms.append(('svd', TruncatedSVD(n_components=7)))
# create the feature union
fu = FeatureUnion(transforms)

# define the feature selection
rfe = RFE(estimator=LogisticRegression(solver='liblinear'), n_features_to_select=15)

# define the model
model = LogisticRegression(solver='liblinear')
#model = SVC(kernel='rbf', gamma='scale', C=1.0)
#model = RandomForestClassifier()

# define the pipeline
steps = list()
steps.append(('fu', fu))
steps.append(('rfe', rfe))
steps.append(('m', model))
pipeline = Pipeline(steps=steps)
# define the cross-validation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

NameError: name 'MinMaxScaler' is not defined